### Importing Libraries

In [2]:
from pyspark.sql import SparkSession, functions as func 
from pyspark.sql.types import IntegerType

### Loading Files

In [3]:
spark = SparkSession.builder.appName("GooglePlayStore").getOrCreate()
df = spark.read.load("google_apps.csv", format='csv', sep=',', header='true', escape='"', inferschema='true')


24/02/11 16:26:13 WARN Utils: Your hostname, Poojas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.20.10.2 instead (on interface en0)
24/02/11 16:26:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/11 16:26:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/02/11 16:26:25 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
# Total Records:
df.count()

9659

In [5]:
# First 10 records
df.show(10)

+---+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|_c0|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+---+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|  0|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|19.0|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|
|  1| Coloring book moana|ART_AND_DESIGN|   3.9|    967|14.0|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|
|  2|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 8.7| 5,000,000+|Free|    0|      Everyone|        Art & Design|    August 1, 2018| 

24/02/11 16:27:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , App, Category, Rating, Reviews, Size, Installs, Type, Price, Content Rating, Genres, Last Updated, Current Ver, Android Ver
 Schema: _c0, App, Category, Rating, Reviews, Size, Installs, Type, Price, Content Rating, Genres, Last Updated, Current Ver, Android Ver
Expected: _c0 but found: 
CSV file: file:///Users/poojasingh/Documents/Git_Reposit/DataEnggProjects/GooglePlaystore_Spark/google_apps.csv


In [6]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: double (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



### Data Cleaning

In [9]:
# Removing Irrelvant Columns
df = df.drop("_c0", "Size", "Content Rating", "Android Ver", "Current Ver", "Last Updated")

In [10]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [11]:
# Converting Columns Data Types
df = df.withColumn("Reviews", func.col("Reviews").cast(IntegerType())) \
        .withColumn("Installs", func.regexp_replace(func.col("Installs"), "[^0-9]", "")) \
        .withColumn("Installs", func.col("Installs").cast(IntegerType())) \
        .withColumn("Price", func.regexp_replace(func.col("Price"), "[$]", "")) \
        .withColumn("Price", func.col("Price").cast(IntegerType()))

In [12]:
df.show(10)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|   50000|Free|    0|        Art & Design|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178|   50000|Free|    0|        Art & Design|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 1000000

### Google Playstore Analysis

In [13]:
df.createOrReplaceTempView("google_apps")

In [15]:
result = spark.sql("SELECT * FROM google_apps LIMIT 10")
result.show()

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|   50000|Free|    0|        Art & Design|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178|   50000|Free|    0|        Art & Design|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 1000000

In [16]:
# Top 10 Most Reviewed Apps
top10_reviewed_apps = spark.sql("SELECT App, sum(Reviews) total_reviews FROM google_apps GROUP BY 1 ORDER BY 2 DESC LIMIT 10")
top10_reviewed_apps.show()

+--------------------+-------------+
|                 App|total_reviews|
+--------------------+-------------+
|            Facebook|     78158306|
|  WhatsApp Messenger|     69119316|
|           Instagram|     66577313|
|Messenger – Text ...|     56642847|
|      Clash of Clans|     44891723|
|Clean Master- Spa...|     42916526|
|      Subway Surfers|     27722264|
|             YouTube|     25655305|
|Security Master -...|     24900999|
|        Clash Royale|     23133508|
+--------------------+-------------+



In [17]:
# Top 10 Most Installed Apps
top10_installed_apps = spark.sql("SELECT App, sum(Installs) total_installed FROM google_apps GROUP BY 1 ORDER BY 2 DESC LIMIT 10")
top10_installed_apps.show()

+--------------------+---------------+
|                 App|total_installed|
+--------------------+---------------+
|  Google Street View|     1000000000|
|         Google News|     1000000000|
|      Subway Surfers|     1000000000|
|   Google Play Books|     1000000000|
|           Instagram|     1000000000|
|  WhatsApp Messenger|     1000000000|
|Google Play Movie...|     1000000000|
|              Google|     1000000000|
|               Gmail|     1000000000|
|   Google Play Games|     1000000000|
+--------------------+---------------+



In [18]:
# Categorywise App Distribution
categorywise_apps = spark.sql("SELECT Category, sum(Installs) total_installed FROM google_apps GROUP BY 1 ORDER BY 2 DESC LIMIT 10")
categorywise_apps.show()

+------------------+---------------+
|          Category|total_installed|
+------------------+---------------+
|              GAME|    13878924415|
|     COMMUNICATION|    11038276251|
|             TOOLS|     8001771915|
|      PRODUCTIVITY|     5793091369|
|            SOCIAL|     5487867902|
|       PHOTOGRAPHY|     4649147655|
|            FAMILY|     4427941505|
|     VIDEO_PLAYERS|     3926902720|
|  TRAVEL_AND_LOCAL|     2894887146|
|NEWS_AND_MAGAZINES|     2369217760|
+------------------+---------------+



In [20]:
# Top 10 Paid Apps
top10_paidapps = spark.sql("SELECT App, sum(Price) price FROM google_apps WHERE Type = 'Paid' GROUP BY 1 ORDER BY 2 DESC LIMIT 10")
top10_paidapps.show()

+--------------------+-----+
|                 App|price|
+--------------------+-----+
|I'm Rich - Trump ...|  400|
|most expensive ap...|  399|
|          I am Rich!|  399|
|      I am Rich Plus|  399|
|I'm Rich/Eu sou R...|  399|
|   I Am Rich Premium|  399|
|           I am Rich|  399|
|  I AM RICH PRO PLUS|  399|
|       I Am Rich Pro|  399|
|  I am rich(premium)|  399|
+--------------------+-----+



In [22]:
# Top 10 Paid Ratings App
top10_paid_ratings = spark.sql("SELECT App, max(Rating) FROM google_apps WHERE Type = 'Paid' GROUP BY 1 ORDER BY 2 DESC LIMIT 10")
top10_paid_ratings.show()

+--------------------+-----------+
|                 App|max(Rating)|
+--------------------+-----------+
|Super Hearing Sec...|        5.0|
|CI 174 Gray Icon ...|        5.0|
|   AJ Blue Icon Pack|        5.0|
|AJ Gray Dark Icon...|        5.0|
|             Mu.F.O.|        5.0|
|      FHR 5-Tier 2.0|        5.0|
|30WPM Amateur ham...|        5.0|
|Hey AJ! It's Bedt...|        5.0|
|     P-Home for KLWP|        5.0|
|        Morse Player|        5.0|
+--------------------+-----------+

